# Quantum Neural Network (QNN)

The key different of QNN from QSVM.Variational is as follows:
- QSVM.Variational needs n qubits to encode n-dim datapoints
- QNN needs log2(n) qubits to encode n-dim datapoints
This means QNN has the potential high-dimensional datapoints with a limited number of qubits. 


### Technical design and comparison
Below shows our proposal:
![structure](QNN.png)

Given the 4-d data point, [x1, x2, x3, x4], we use 2 qubits to encode it. For simplicity, let us assume the data point is normalized during the preprocessing. 

We first invoke the custom initialization algorithm, which creates the circuit that produces the amplitude vector (i.e., the normalized data point); We then apply the variational form, similar to QSVM.Variational. Lastly, we derive the cost function. The optimizer is applied to minimize the cost function. 

In contrast, QSVM.Varitional needs 4 qubits. Note that QSVM relies on the proper feature map, which requires some domain expertise to design. 

### Result
We applied QNN and QSVM.Variational to the Wine dataset. QNN has much better accuracy (90%) than QSVM.Vartional (around 50%). Besides, QNN is 3X faster for this dataset. 

In the following, we will reproduce the result.

We will use the Wine dataset:

In [1]:

import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA


def Wine(training_size, test_size, n):
    class_labels = [r'A', r'B', r'C']

    data, target = datasets.load_wine(True)
    sample_train, sample_test, label_train, label_test = train_test_split(data, target, test_size=test_size, random_state=7)

    # Now we standarize for gaussian around 0 with unit variance
    std_scale = StandardScaler().fit(sample_train)
    sample_train = std_scale.transform(sample_train)
    sample_test = std_scale.transform(sample_test)

    # Now reduce number of features to number of qubits
    pca = PCA(n_components=n).fit(sample_train)
    sample_train = pca.transform(sample_train)
    sample_test = pca.transform(sample_test)

    # Scale to the range (-1,+1)
    samples = np.append(sample_train, sample_test, axis=0)
    minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
    sample_train = minmax_scale.transform(sample_train)
    sample_test = minmax_scale.transform(sample_test)
    # Pick training size number of samples from each distro
    training_input = {key: (sample_train[label_train == k, :])[:training_size] for k, key in enumerate(class_labels)}
    test_input = {key: (sample_train[label_train == k, :])[training_size:(
        training_size+test_size)] for k, key in enumerate(class_labels)}
    return sample_train, training_input, test_input, class_labels

Next, let us import the necessary dependency:

In [2]:
import numpy as np
import scipy

from qiskit import BasicAer
from qiskit.aqua.input import SVMInput
from qiskit.aqua import run_algorithm, QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QSVMVariational
from qiskit.aqua.components.optimizers import SPSA, COBYLA
from qiskit.aqua.components.feature_maps import SecondOrderExpansion
from qiskit.aqua.components.variational_forms import RYRZ, RY

Let us run QNN against Wine dataset first. 

In [11]:
feature_dim = 4 # dimension of each data point
training_dataset_size = 20
testing_dataset_size = 10
random_seed = 10598
np.random.seed(random_seed)

sample_Total, training_input, test_input, class_labels = Wine(training_size=training_dataset_size,
                                                                     test_size=testing_dataset_size,
                                                                     n=feature_dim)
svm_input = SVMInput(training_input, test_input)
params = {
    'problem': {'name': 'svm_classification', 'random_seed': random_seed},
    'algorithm': {'name': 'QNN'},
    'backend': {'provider': 'qiskit.BasicAer', 'name': 'statevector_simulator'},
    'optimizer': {'name': 'COBYLA', 'maxiter':200},
    'variational_form': {'name': 'RYRZ', 'depth': 3}
}
result = run_algorithm(params, svm_input)
print("testing accuracy: ", result['testing_accuracy'])        

testing accuracy:  0.8666666666666667


Let us run QSVM.Variational against Wine now. 

In [12]:
feature_dim = 4 # dimension of each data point
training_dataset_size = 20
testing_dataset_size = 10
random_seed = 10598
np.random.seed(random_seed)

params = {
    'problem': {'name': 'svm_classification', 'random_seed': random_seed},
    'algorithm': {'name': 'QSVM.Variational'}, #
    'backend': {'provider': 'qiskit.BasicAer', 'name': 'statevector_simulator'},
    'optimizer': {'name': 'COBYLA', 'maxiter':200},
    'variational_form': {'name': 'RYRZ', 'depth': 3}
}
result = run_algorithm(params, svm_input)
print("testing accuracy: ", result['testing_accuracy'])     

testing accuracy:  0.5
